In [31]:
# mmdetection이 도출한 segm.json 파일을 기반으로
# 이미지 segmentation 진행

# Import libraries
import numpy as np
import cv2
import json
import pycocotools.mask as mask

# Read the annotations
file_path = "./mmdetection/gwang_test.segm.json"
with open(file_path, 'r') as f:
    data = json.load(f)

updated_data = []

# For each annotation
for annotation in data:

    # Initialize variables
    obj = {}
    segmentation = []
    segmentation_polygons = []

    # Decode the binary mask
    mask_list = mask.decode(annotation['segmentation'])
    mask_list = np.ascontiguousarray(mask_list, dtype=np.uint8)
    contours, hierarchy = cv2.findContours((mask_list).astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
 
    # Get the contours
    for contour in contours:
        contour = contour.flatten().tolist()
        segmentation.append(contour)
        if len(contour) > 4:
            segmentation.append(contour)
    if len(segmentation) == 0:
        continue

    # Get the polygons as (x, y) coordinates
    for i, segment in enumerate(segmentation):
        poligon = []
        poligons = []
        for j in range(len(segment)):
            poligon.append(segment[j])
            if (j+1)%2 == 0:
                poligons.append(poligon)
                poligon = []
        segmentation_polygons.append(poligons)

    # Save the segmentation and polygons for the current annotation
#     obj['segmentation'] = segmentation
    obj['image_id'] = annotation['image_id']
    obj['segmentation_polygons'] = segmentation_polygons
    updated_data.append(obj)

In [91]:
check = []
for i in range(len(updated_data)):
    check.append(updated_data[i]['image_id'])

In [62]:
# for i in range(len(data)):
#     print(data[i]['image_id'])
# print(data[5])
# print(len(data))
# print(len(updated_data))

# count = 0
# for i in range(len(updated_data)):
#     if len(updated_data[i]['segmentation_polygons']) != 2 :
#         count += 1
# print(count)
# 23226-1985

# updated_data[0]['segmentation_polygons']
np.array(updated_data[0]['segmentation_polygons'])
# updated_data[4]

array([[[144, 141],
        [143, 142],
        [141, 142],
        ...,
        [216, 142],
        [208, 142],
        [207, 141]],

       [[144, 141],
        [143, 142],
        [141, 142],
        ...,
        [216, 142],
        [208, 142],
        [207, 141]]])

In [15]:
# 흰배경 jpg 만들기
# 흰배경을 만들 때는 아래 해당 코드로

img = cv2.imread('./data/color_image/'+str(updated_data[125]['image_id'])+'.jpg')
height = img.shape[0]
width = img.shape[1]
mask = np.zeros((height, width), img.dtype) # mask 생성
points = np.array(updated_data[125]['segmentation_polygons'][0]).reshape((1,) + np.array(updated_data[125]['segmentation_polygons'][0]).shape)
# points -> 채워진 다각형을 그리는 fillPoly에 사용될 point 좌표 
cv2.fillPoly(mask, points, (255)) # mask에 해당하는 이미지 영역을 흰색으로 채운 다각형 생성
res = cv2.bitwise_and(img,img,mask = mask) # mask에 해당하는 영역이면 1, 아니면 0으로 해서 bitwise 연산

# backgroud color -> white
bg = np.ones_like(res, np.uint8)*255
cv2.bitwise_not(bg,bg, mask=mask)
res2 = bg+ res

rect = cv2.boundingRect(points) # returns (x,y,w,h) of the rect
cropped = res2[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]

cv2.imwrite('./data/' + str(updated_data[125]['image_id'])+'.jpg', cropped)
cv2.waitKey(0)

-1

In [ ]:
# 1.jpg 만들기
img = cv2.imread('./data/color_image/'+str(updated_data[0]['image_id'])+'.jpg')
height = img.shape[0]
width = img.shape[1]
mask = np.zeros((height, width), dtype=np.uint8)
points = np.array(updated_data[0]['segmentation_polygons'][0]).reshape((1,) + np.array(updated_data[0]['segmentation_polygons'][0]).shape)
cv2.fillPoly(mask, points, (255))
res = cv2.bitwise_and(img,img,mask = mask)
rect = cv2.boundingRect(points) # returns (x,y,w,h) of the rect
cropped = res[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]
cv2.imwrite('./data/' + str(updated_data[0]['image_id'])+'.jpg', cropped)
cv2.waitKey(0)

In [3]:
# 1.jpg를 제외하고 의상 탐지 및 분류 첫번째 사진을 그 색으로 가정하고 crop

img_id = 1
for i in range(1,len(updated_data)):
    img_id =  updated_data[i]['image_id']
    this_id = updated_data[i-1]['image_id']
    if this_id == img_id:
        continue
    img = cv2.imread('./data/color_image/'+str(updated_data[i]['image_id'])+'.jpg')
    height = img.shape[0]
    width = img.shape[1]
    mask = np.zeros((height, width), dtype=np.uint8)
    points = np.array(updated_data[i]['segmentation_polygons'][0]).reshape((1,) + np.array(updated_data[i]['segmentation_polygons'][0]).shape)
    cv2.fillPoly(mask, points, (255))
    res = cv2.bitwise_and(img,img,mask = mask)
    rect = cv2.boundingRect(points) # returns (x,y,w,h) of the rect
    cropped = res[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]
    cv2.imwrite('./data/cropped_color_image_add/' + str(updated_data[i]['image_id'])+'.jpg', cropped)
    cv2.waitKey(0)